## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [23]:
attrition_df['Department'].value_counts()

Department
Research & Development    961
Sales                     446
Human Resources            63
Name: count, dtype: int64

In [3]:
#Drop null values
attrition_df = attrition_df.dropna(axis=1)


In [4]:
# Create y_df with the Attrition and Department columns
Y_df = attrition_df[['Attrition', 'Department']]
Y_df.head()

,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [5]:
# Create a list of at least 10 column names to use as X data
column_names = ['Age','BusinessTravel', 'DistanceFromHome', 'Education', 'EducationField', 'EnvironmentSatisfaction', 'HourlyRate', 'PerformanceRating', 'RelationshipSatisfaction', 'TotalWorkingYears']


# Create X_df using your selected columns
X_df = pd.DataFrame(attrition_df[column_names])

# Show the data types for X_df
X_df.dtypes


Age                          int64
BusinessTravel              object
DistanceFromHome             int64
Education                    int64
EducationField              object
EnvironmentSatisfaction      int64
HourlyRate                   int64
PerformanceRating            int64
RelationshipSatisfaction     int64
TotalWorkingYears            int64
dtype: object

In [6]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_df, Y_df, test_size = 0.2, random_state=42)


In [7]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
#Encoding the BusinessTravel with OrdinalEncoder as there is a heirarchy to preserve in the data
ordinal_encoder = OrdinalEncoder()
X_train['BusinessTravelEncoded'] = ordinal_encoder.fit_transform(X_train[['BusinessTravel']])
X_test['BusinessTravelEncoded'] = ordinal_encoder.fit_transform(X_test[['BusinessTravel']])
X_train = X_train.drop('BusinessTravel', axis=1)
X_test = X_test.drop('BusinessTravel', axis=1)

In [8]:
#Encode BusinessField with OneHotEncoder as the feature types are unrelated
#Encode BusinessField with OneHotEncoder as the feature types are unrelated
one_hot_encoder = OneHotEncoder()
EducationField_X_train = one_hot_encoder.fit_transform(X_train[['EducationField']])
EducationField_X_test = one_hot_encoder.fit_transform(X_test[['EducationField']])

# Get the correct feature names from the one-hot encoder
column_names = one_hot_encoder.get_feature_names_out(['EducationField'])

EducationFieldEncoded_X_train_df = pd.DataFrame(EducationField_X_train.toarray(), columns=column_names) # Convert to dense array
EducationFieldEncoded_X_test_df = pd.DataFrame(EducationField_X_test.toarray(), columns=column_names) # Convert to dense array

X_train = pd.concat([X_train, EducationFieldEncoded_X_train_df], axis=1)
X_test = pd.concat([X_test, EducationFieldEncoded_X_test_df], axis=1)

# Fix the typo in the drop method and remove the unnecessary quotes
X_train = X_train.drop('EducationField', axis=1)
X_test = X_test.drop('EducationField', axis=1)

In [9]:
from re import X
# Create a StandardScaler
standard_scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_train_scaled = standard_scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = standard_scaler.transform(X_train)
X_test_scaled = standard_scaler.transform(X_test)


In [10]:
# Create a OneHotEncoder for the Department column
department_encoder = OneHotEncoder()

# Fit the encoder to the training data and transform both
# training and testing data
Department_Y_train = department_encoder.fit_transform(Y_train[['Department']])
Department_Y_test = department_encoder.transform(Y_test[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
department_column_names = department_encoder.get_feature_names_out(['Department'])
DepartmentEncoded_Y_train_df = pd.DataFrame(Department_Y_train.toarray(), columns=department_column_names) # Convert to dense array
DepartmentEncoded_Y_test_df = pd.DataFrame(Department_Y_test.toarray(), columns=department_column_names) # Convert to dense array

#Concatenate the encoded features to the train and test data frame
Y_train_department = pd.concat([Y_train, DepartmentEncoded_Y_train_df], axis=1)
Y_test_department = pd.concat([Y_test, DepartmentEncoded_Y_test_df], axis=1) # Using the correct variable name now

# Drop the unencoded Department column
Y_train_department = Y_train_department.drop(['Department', 'Attrition'], axis=1)
Y_test_department = Y_test_department.drop(['Department', 'Attrition'], axis=1)

In [11]:

y_department_train = Y_train_department.to_numpy()
y_department_test = Y_test_department.to_numpy()


In [12]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder()

# Fit the encoder to the training data
Attrition_Y_train = attrition_encoder.fit_transform(Y_train[['Attrition']])
Attrition_Y_test = attrition_encoder.fit_transform(Y_test[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
attrition_column_names = attrition_encoder.get_feature_names_out(['Attrition'])
AttritionEncoded_Y_train_df = pd.DataFrame(Attrition_Y_train.toarray(), columns=attrition_column_names) # Convert to dense array
AttritionEncoded_Y_test_df = pd.DataFrame(Attrition_Y_test.toarray(), columns=attrition_column_names) # Convert to dense array

#Concatenate the encoded features to the train and test data frame
Y_train_attrition = pd.concat([Y_train, AttritionEncoded_Y_train_df], axis=1)
Y_test_attrition = pd.concat([Y_test, AttritionEncoded_Y_test_df], axis=1)

# Drop the unencoded Attrition column
Y_train_attrition = Y_train_attrition.drop(['Attrition', 'Department'], axis=1)
Y_test_attrition = Y_test_attrition.drop(['Attrition', 'Department'], axis=1)



In [13]:
y_attrition_train = Y_train_attrition.to_numpy()
y_attrition_test = Y_test_attrition.to_numpy()


## Create, Compile, and Train the Model

In [14]:
# Find the number of columns in the X training data
X_train_scaled.shape[1]
# Create the input layer
input_layer = layers.Input(shape=(X_train_scaled.shape[1],), name='input_features')
# Create the hidden layer

# Create at least two shared layers
shared_layer1 = layers.Dense(units=128, activation='softmax')(input_layer)
shared_layer2 = layers.Dense(units=64, activation='softmax')(shared_layer1)

# Create the output layer

In [15]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden = layers.Dense(units=4, activation='softmax', name='department_hidden')(shared_layer2)

# Create the output layer
# Changed the name of the output layer to 'department_output'
department_output = layers.Dense(units=3, activation='softmax', name='department_output')(department_hidden)



In [16]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
# Changed the name of the hidden layer to 'attrition_hidden'
attrition_hidden = layers.Dense(units=4, activation='softmax', name='attrition_hidden')(shared_layer2)

# Create the output layer
attrition_output = layers.Dense(units=2, activation='softmax', name='attrition_output')(attrition_hidden)


In [17]:
# Create the model
model = Model(inputs=input_layer, outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
              loss={'department_output': 'categorical_crossentropy', 'attrition_output': 'binary_crossentropy'},
              metrics={'department_output': 'accuracy', 'attrition_output': 'accuracy'})

# Summarize the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_features (InputLayer  [(None, 15)]                 0         []                            
 )                                                                                                
                                                                                                  
 dense (Dense)               (None, 128)                  2048      ['input_features[0][0]']      
                                                                                                  
 dense_1 (Dense)             (None, 64)                   8256      ['dense[0][0]']               
                                                                                                  
 department_hidden (Dense)   (None, 4)                    260       ['dense_1[0][0]']         

In [21]:
# Train the model
model.fit(
    X_train_scaled,
    {'department_output': y_department_train, 'attrition_output': y_attrition_train},
    epochs=20,
    batch_size=32,
    validation_split=0.2
)


Epoch 1/200
36/36 [==============================] - 0s 5ms/step - loss: nan - department_output_loss: nan - attrition_output_loss: nan - department_output_accuracy: 0.2386 - attrition_output_accuracy: 0.8715 - val_loss: nan - val_department_output_loss: nan - val_attrition_output_loss: nan - val_department_output_accuracy: 0.0810 - val_attrition_output_accuracy: 0.8169
Epoch 2/200
36/36 [==============================] - 0s 4ms/step - loss: nan - department_output_loss: nan - attrition_output_loss: nan - department_output_accuracy: 0.2386 - attrition_output_accuracy: 0.8715 - val_loss: nan - val_department_output_loss: nan - val_attrition_output_loss: nan - val_department_output_accuracy: 0.0810 - val_attrition_output_accuracy: 0.8169
Epoch 3/200
36/36 [==============================] - 0s 5ms/step - loss: nan - department_output_loss: nan - attrition_output_loss: nan - department_output_accuracy: 0.2386 - attrition_output_accuracy: 0.8715 - val_loss: nan - val_department_output_loss:

In [22]:
# Evaluate the model with the testing data
test_results = model.evaluate(X_test_scaled, {'department_output': y_department_test, 'attrition_output': y_attrition_test})
test_results

17/17 [==============================] - 0s 4ms/step - loss: nan - department_output_loss: nan - attrition_output_loss: nan - department_output_accuracy: 0.4738 - attrition_output_accuracy: 0.9270


[nan, nan, nan, 0.4737827777862549, 0.9269663095474243]

In [20]:
# Print the accuracy for both department and attrition
print(f"Department Accuracy: {test_results[3]}")
print(f"Attrition Accuracy: {test_results[4]}")

Department Accuracy: 0.4737827777862549
Attrition Accuracy: 0.9269663095474243


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not? The Department target is imbalance and Precision and Recall may do a better job than accuracy to know which targets the model is predicting accurately

2. What activation functions did you choose for your output layers, and why?  I tried RELU in the shared hidden layers in original run of the model and Softmax for the final branched layers.  Softmax is the obvious choice for "Department" as the choice isn't binary, even though the model did a terrible job predicting it. I changed the activation function in the shared layers to Softmax and the model didn't improve.

3. Can you name a few ways that this model might be improved?  The model may be improved by balancing the data in the "Department" target. It could also potential improve by adding additional hidden layers.

YOUR ANSWERS HERE

1.
2.
3.